# Download and process Pelletier soil data set into geotiffs at native resolution

In [1]:
import os
import sys
import shutil
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling                                                                                        

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
code_path            = cs.read_from_config(config_file,'code_path')
data_path            = cs.read_from_config(config_file,'data_path')
geospatial_temp_path = cs.read_from_config(config_file,'geospatial_temp_path')
soil_path            = cs.read_from_config(config_file,'soil_depth_path')
soil_url_file        = cs.read_from_config(config_file,'soil_depth_urls')
download_area        = cs.read_from_config(config_file,'geospatial_area')

### Download data

In [4]:
download_folder = Path(data_path) / geospatial_temp_path / soil_path / 'download'

In [5]:
download_folder.mkdir(parents=True, exist_ok=True)

In [10]:
with open(Path(code_path)/soil_url_file) as url_file:
    for url in url_file:
        cs.download_url_into_folder(url,download_folder)

Successfully downloaded https://daac.ornl.gov/daacdata/global_soil/Global_Soil_Regolith_Sediment/data/average_soil_and_sedimentary-deposit_thickness.tif

Successfully downloaded https://daac.ornl.gov/daacdata/global_soil/Global_Soil_Regolith_Sediment/data/hill-slope_valley-bottom.tif

Successfully downloaded https://daac.ornl.gov/daacdata/global_soil/Global_Soil_Regolith_Sediment/data/land_cover_mask.tif

Successfully downloaded https://daac.ornl.gov/daacdata/global_soil/Global_Soil_Regolith_Sediment/data/upland_hill-slope_regolith_thickness.tif

Successfully downloaded https://daac.ornl.gov/daacdata/global_soil/Global_Soil_Regolith_Sediment/data/upland_hill-slope_soil_thickness.tif

Successfully downloaded https://daac.ornl.gov/daacdata/global_soil/Global_Soil_Regolith_Sediment/data/upland_valley-bottom_and_lowland_sedimentary_deposit_thickness.tif


In [5]:
# Track files
contents = os.listdir(download_folder)
src_files = [download_folder/file for file in contents]

### Subset files to domain

In [6]:
des_folder = Path(data_path) / geospatial_temp_path / soil_path / 'raw'

In [7]:
des_folder.mkdir(parents=True, exist_ok=True)

In [8]:
# Convert subsetting area into a usable GDAL setting
# subset_area = [lon_min, lon_max, lat_min, lat_max]
# GDAL window = [ulx, uly, lrx, lry]; [upper left x, upper left y, lower right x, lower right y]
# Mapping:
#   ulx = lon_min = subset_area[0]
#   uly = lat_max = subset_area[3]
#   lrx = lon_max = subset_area[1]
#   lry = lat_min = subset_area[2]
subset_coor = download_area.split(',')
window = [subset_coor[0], subset_coor[3], subset_coor[1], subset_coor[2]]

In [10]:
for src_file in src_files:
    des_file = des_folder/os.path.basename(src_file)
    cs.subset_tif(src_file,des_file,window)

### Delete the individual GeoTIFF files to save space

In [11]:
if os.path.isdir(download_folder): 
    shutil.rmtree(download_folder)